In [1]:
import urllib2
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from urllib import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list
import time
import random

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]


### Pseudo Code:
While date < end_date:
    for each article link, if company in list_of_companies is in link:
        pull the [Company_Name, Date], and open link:
            pull Earning_Call.

Final Output : [Company_Name (str), Date (date-time), Earning_Call (str)]

Then:
Run sentiment analysis to make it : [Company_Name (str), Date (date-time), Earning_Call_Score (str)] # Remove the Call to reduce filesize


company_input_data = pd.read_csv('Mutual_Funds_companies.csv', index_col = 0)
company_input_data['name of issuer'].unique()

nasdaqdf = pd.read_csv('NASDAQ.csv')
nysedf = pd.read_csv('NYSE.csv')
amexdf = pd.read_csv('AMEX.csv')

total_symbols = amexdf.append(nasdaqdf.append(nysedf))

name_original = []
name_new = []
symbol_new = []

lower1 = [company_input_data['name of issuer'].unique()[x].lower() for x in range(len(company_input_data['name of issuer'].unique()))]
lower2 = [total_symbols.Name.iloc[x].lower() for x in range(len(total_symbols))]
for comp in lower1:
    for ref in lower2:
        if comp in ref:
            name_original.append(comp)
            name_new.append(ref)
            symbol_new.append(total_symbols[total_symbols.Name.str.lower() == ref].Symbol.str[0:])

In [2]:
df_of_companies = pd.read_csv('Company_list_final.csv', index_col=0) # < This should be the stock symbol list of companies we are interested in.
df_of_companies.columns = ['Symbol']
list_of_companies = [x for x in df_of_companies.Symbol] + ['GM']
list_of_companies


['DDD',
 'MMM',
 'ABT',
 'ANF',
 'ACN',
 'BHVN',
 'APB',
 'ATVI',
 'AYI',
 'ATGE',
 'ADTN',
 'AAP',
 'AMD',
 'ACM',
 'AES',
 'AET',
 'AMG',
 'AFL',
 'AGCO',
 'AEM',
 'AGU',
 'AL',
 'ALK',
 'ALB',
 'AA',
 'ALEX',
 'ARE',
 'Y',
 'AKP',
 'LNT',
 'AHPI',
 'MDRX',
 'ALL',
 'ALLY',
 'MO',
 'AMCX',
 'DOX',
 'UHAL',
 'AAL',
 'AAT',
 'AEO',
 'AXP',
 'AMP',
 'ABC',
 'AMGN',
 'AFSI-PA',
 'AEUA',
 'ADI',
 'NLY',
 'ANSS',
 'ANTM',
 'APA',
 'ARI',
 'AIY',
 'APLE',
 'AMAT',
 'ATR',
 'WTR',
 'ARMK',
 'ARH-PC.CL',
 'ADM',
 'AWI',
 'HRS',
 'ARW',
 'AJG',
 'APAM',
 'ASNA',
 'ASH',
 'ASB',
 'AGO',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AVY',
 'CAR',
 'AVT',
 'BHGE',
 'BLL',
 'BAC',
 'BOH',
 'BXS',
 'BBG',
 'BAX',
 'AHC',
 'BDX',
 'BBBY',
 'BDC',
 'WRB',
 'BRK.A',
 'BBY',
 'BHP',
 'BIG',
 'ABMD',
 'BIO',
 'BKH',
 'BAF',
 'BOBE',
 'BA',
 'BOKF',
 'BAH',
 'BWA',
 'BPMP',
 'BDN',
 'CBM',
 'EAT',
 'BMY',
 'BR',
 'BRO',
 'BC',
 'BG',
 'CHRW',
 'CMA',
 'CBT',
 'COG',
 'CADE',
 'CPN',
 'CPT',
 'JCTCF',
 'CPB',
 'ACC',


In [111]:
Company_Name = []
Date = []
Earning_Call_list = []

def crawl_this_url(url, list_of_companies):
    Company_Name = []
    Date = []
    Earning_Call = []
    
    class MyOpener(FancyURLopener, object):
        version = choice(user_agents)
    myopener = MyOpener()

    html = myopener.open(url).read()
    time.sleep(random.randint(2,7))
    soup = BeautifulSoup(html, "lxml")

    name_index = -1
    name_for_href = soup.find_all('a', class_ = 'dashboard-article-link')
    for name in soup.find_all('a', sasource = 'earnings-center-transcripts_symbol'):
        name_index += 1
        for company_name in list_of_companies:
            if str(company_name) == name.text: #Checks if the 'name' iterator contains any of the listed companies
                
                #### Append Company Name ####
                Company_Name.append(company_name)

                #### Obatin and Append Date ####
                date = str(soup.find_all('div', class_ = 'article-desc')[name_index])
                date_start = date.find('<span class="bullet">•</span>')
                remaining = date[date_start:]
                date_end = remaining.find('<span>•</span>')
                date_as_str = remaining[len('<span class="bullet">•</span>'):date_end]
                Date.append(date_as_str.strip())

                #### Go to link, and obtain Call ####
                earning_call = ''
                inner_page_number = 1
                base_inner_page = 'https://seekingalpha.com' + name_for_href[name_index].get('href')
                inner_page = myopener.open(base_inner_page).read()
                #time.sleep(random.randint(5,15))
                inner_soup = BeautifulSoup(inner_page, 'lxml')

                # Iterate over each page, and obtain the call #
                for inner_text in inner_soup.find_all('p', class_ = re.compile('p p')):
                    earning_call += inner_text.text
                Earning_Call.append(earning_call)
    return Company_Name, Date, Earning_Call




def get_ecalls(page_number, list_of_companies):
    url = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_number)
    try:
        Company_Nameout, Dateout, Earning_Callout = crawl_this_url(url,list_of_companies)
        Company_Name_list += Company_Nameout
        Date_list += Dateout
        Earning_Call_list += Earning_Callout
        print(page_number, 'Worked')
        
    except:
        print(page_number, 'Sleep')
        time.sleep(random.randint(5,15))
        get_ecalls(page_number, list_of_companies)
    get_ecalls(page_number + 1, list_of_companies)

for page_number in range(128,132):
    url = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_number)
    try:
        Company_Nameout, Dateout, Earning_Callout = crawl_this_url(url,list_of_companies)
        Company_Name_list += Company_Nameout
        Date_list += Dateout
        Earning_Call_list += Earning_Callout
        print(page_number, 'Worked')
    except:
        print(page_number, 'Sleep')
        time.sleep(random.randint(5,15))
        Company_Nameout, Dateout, Earning_Callout = crawl_this_url(url,list_of_companies)
        Company_Name_list += Company_Nameout
        Date_list += Dateout
        Earning_Call_list += Earning_Callout        
        print(page_number, 'Worked')

for i in range(1,1000):
    get_ecalls(i, list_of_companies)

In [3]:
Company_Name = []
Date = []
Earning_Call = []
NameText = []

In [42]:
url = 'https://seekingalpha.com/earnings/earnings-call-transcripts'
class MyOpener(FancyURLopener, object):
    version = choice(user_agents)
myopener = MyOpener()
html = myopener.open(url).read()
soup = BeautifulSoup(html, "lxml")

page_number = 1
target_page = 1000
while page_number <= target_page and ('Access to this page has been denied' not in html):
    print(page_number)
    class MyOpener(FancyURLopener, object):
        version = choice(user_agents)
    myopener = MyOpener()
    
    page_number += 1
    html = myopener.open(url).read()
    time.sleep(random.randint(2,15))
    soup = BeautifulSoup(html, "lxml")

    name_index = -1
    list_of_possible_companies = [x for x in list_of_companies if x in html]
    name_for_href = soup.find_all('a', class_ = 'dashboard-article-link')
    for name in soup.find_all('a', sasource = 'earnings-center-transcripts_symbol'):
        name_index += 1
        for company_name in list_of_possible_companies:
            if str(company_name) == name.text: #Checks if the 'name' iterator contains any of the listed companies
                #### Append Company Name ####
                Company_Name.append(company_name)

                #### Obatin and Append Date ####
                date = str(soup.find_all('div', class_ = 'article-desc')[name_index])
                date_start = date.find('<span class="bullet">•</span>')
                remaining = date[date_start:]
                date_end = remaining.find('<span>•</span>')
                date_as_str = remaining[len('<span class="bullet">•</span>'):date_end]
                Date.append(date_as_str.strip())

                #### Go to link, and obtain Call ####
                earning_call = ''
                inner_page_number = 1
                base_inner_page = 'https://seekingalpha.com' + name_for_href[name_index].get('href')
                inner_page = myopener.open(base_inner_page).read()
                #time.sleep(random.randint(5,15))
                inner_soup = BeautifulSoup(inner_page, 'lxml')

                # Iterate over each page, and obtain the call #
                for inner_text in inner_soup.find_all('p', class_ = re.compile('p p')):
                    earning_call += inner_text.text
                Earning_Call.append(earning_call)
                
        url = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_number)


1
2
3
4
5
6
7
8
9


In [31]:
list_of_html = []

In [32]:
url = 'https://seekingalpha.com/earnings/earnings-call-transcripts'
class MyOpener(FancyURLopener, object):
    version = choice(user_agents)
myopener = MyOpener()
html = myopener.open(url).read()
soup = BeautifulSoup(html, "lxml")

flags = ['permission','denied','access', 'robot']
redo = []

#counter = 1
time_list = list(range(1,3500))
#random.shuffle(time_list)
for page_number in time_list:
 #   print(counter)
    counter += 1
    time.sleep(random.randint(2,7))
    print(page_number)
    class MyOpener(FancyURLopener, object):
        version = choice(user_agents)
    myopener = MyOpener()
    html = myopener.open(url).read()
    if any(x in html.lower() for x in flags):
        print('REDO:',page_number)
        redo.append(page_number)
    list_of_html.append(html)
    url = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_number)

1
('REDO:', 1)
2
('REDO:', 2)


KeyboardInterrupt: 

In [33]:
list_of_html[0]

'<!DOCTYPE html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta charset="utf-8" /><meta content="IE=edge" http-equiv="X-UA-Compatible" /><meta content="width=device-width" name="viewport" /><title>Earnings Call Transcripts | Seeking Alpha</title><meta content="Seeking Alpha" name="application-name" /><meta content="true" name="HandheldFriendly" /><meta content="noodp,noydir" name="robots" /><meta content="Complete transcripts of earnings calls, including the Audio and slideshow presentations." lang="en-US" name="description" /><meta content="https://static.seekingalpha.com/assets/og_image_410-b8960ce31ec84f7f12dba11a09fc1849b69b234e0f5f39d7c62f46f8692e58a5.png" name="thumbnail" /><link rel="canonical" href="https://seekingalpha.com/earnings/earnings-call-transcripts" /><link href="https://seekingalpha.com/earnings/earnings-call-transcripts/2" rel="next" /><link href="https://static.seekingalpha.com/assets/favicon-d8a68714f8b18f200bbdab463556289870b8fc0c008

In [9]:
co = 1
for html in list_of_html:
    print(co) 
    co += 1
    class MyOpener(FancyURLopener, object):
        version = choice(user_agents)
    myopener = MyOpener()
    
    soup = BeautifulSoup(html, "lxml")
    
    name_index = -1
    name_for_href = soup.find_all('a', class_ = 'dashboard-article-link')
    for name in soup.find_all('a', sasource = 'earnings-center-transcripts_symbol'):
        print(name.text)
        name_index += 1
        for company_name in list_of_companies:
            if str(company_name).strip() == name.text.strip(): #Checks if the 'name' iterator contains any of the listed companies
                print('Match', company_name)
                #### Append Company Name ####
                Company_Name.append(company_name)
                NameText.append(name.text)

                #### Obatin and Append Date ####
                date = str(soup.find_all('div', class_ = 'article-desc')[name_index])
                date_start = date.find('<span class="bullet">•</span>')
                remaining = date[date_start:]
                date_end = remaining.find('<span>•</span>')
                date_as_str = remaining[len('<span class="bullet">•</span>'):date_end]
                Date.append(date_as_str.strip())

                #### Go to link, and obtain Call ####
                earning_call = ''
                inner_page_number = 1
                base_inner_page = 'https://seekingalpha.com' + name_for_href[name_index].get('href')
                inner_page = myopener.open(base_inner_page).read()
                #time.sleep(random.randint(5,15))
                inner_soup = BeautifulSoup(inner_page, 'lxml')

                # Iterate over each page, and obtain the call #
                for inner_text in inner_soup.find_all('p', class_ = re.compile('p p')):
                    earning_call += inner_text.text
                Earning_Call.append(earning_call)
    
    

1
EPD
CR
('Match', 'CR')
DLNG
POWL
ZNGA
GALT
DEPO
VRA
BF.B
CHKE
GILD
('Match', 'GILD')
HRB
AEO
('Match', 'AEO')
LEE
JW.A
('Match', 'JW.A')
KLXI
IEC
LAYN
SCWX
ZX
HBAYF
FRED
MU
('Match', 'MU')
VNET
PLAY
RH
BAC
('Match', 'BAC')
MXIM
VEEV
OXM
2
AVAV
BMO
AZO
HQY
IDT
TOL
('Match', 'TOL')
LQDT
JPM
('Match', 'JPM')
EMKR
VZ
('Match', 'VZ')
IBCP
IBM
AET
('Match', 'AET')
WFC
('Match', 'WFC')
T
('Match', 'T')
S
('Match', 'S')
BNED
('Match', 'BNED')
FRAN
LE
MSFT
('Match', 'MSFT')
TTMI
HDS
JILL
GIII
EBAY
('Match', 'EBAY')
CSCO
('Match', 'CSCO')
COUP
CMCSA
ASNA
('Match', 'ASNA')
ZGNX
3
4
KR
('Match', 'KR')
GMLP
DSGX
DL
DCI
('Match', 'DCI')
CELG
('Match', 'CELG')
JACK
GLNG
EXPR
TITN
TNP
MNR
MIK
BKS
PERY
CM
PVH
('Match', 'PVH')
BOSC
LZB
SHLD
GWRE
SNPS
('Match', 'SNPS')
BOX
WDAY
TLYS
SMTC
NWY
HOME
V
('Match', 'V')
OCSL
5
6
7
8
9
10
11
ROST
('Match', 'ROST')
AFAM
BRLAF
NUACF
CNIT
IVTJF
VIVEF
KBCSF
SDXOF
CAPC
BONT
HP
('Match', 'HP')
PYPL
BERY
MDWD
ROGFF
STNG
HMLP
EZPW
('Match', 'EZPW')
BRC
MAGS
GPRK
SPH
S

#### Dates are in a weird format:
Within 2017, they have a "Fri, Dec.  1,  9:56 PM" format. Every other year, they have a "Mar. 15, 2016,  8:03 PM" format

In [99]:
print(len(Earning_Call))

170


In [121]:
list_of_html[1]

'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body bgcolor="white">\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx/1.8.0</center>\r\n</body>\r\n</html>\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n'

In [22]:
Company_Name

['GILD',
 'AEO',
 'JW.A',
 'MU',
 'BAC',
 'TOL',
 'JPM',
 'VZ',
 'AET',
 'WFC',
 'T',
 'S',
 'BNED',
 'MSFT',
 'EBAY',
 'CSCO',
 'ASNA',
 'CSCO',
 'TIME',
 'BIG',
 'EMR',
 'AFL',
 'F',
 'TD',
 'KR',
 'DCI',
 'CELG',
 'PVH',
 'SNPS',
 'V',
 'MSFT',
 'GLW',
 'BMY',
 'TIF',
 'AMD',
 'ADSK',
 'MRVL',
 'NUAN',
 'EBAY',
 'CSCO',
 'MU',
 'CBK',
 'CPRT',
 'HPE']

In [44]:
datadf = pd.DataFrame({'Company' : Company_Name_list, 'Date' : Date_list, 'Earning_Call' : Earning_Call_list})
datadf[datadf.Company == 'MKC']

,Company,Date,Earning_Call
12,MKC,"Thu, Sep. 28, 1:21 PM","McCormick & Company, Inc. (NYSE:MKC)Q3 2017 Re..."


In [23]:
# Take care of the dates problem
datadf.Date = datadf.Date.str[:-10]
datadf.Date[datadf.Date.str[3] == ','] = datadf.Date.str[5:]
datadf.Date[datadf.Date.str.len() < 13] += ', 2017'
datadf.Date[datadf.Date.str.len() == 13] = datadf.Date.str[:3] + datadf.Date.str[4:]
datadf.Date[datadf.Date.str[4] == ' '] = datadf.Date.str[:4] + '0' + datadf.Date.str[5:] 

datadf.Date[datadf.Date.str[:3] == 'Jan'] = '1,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Feb'] = '2,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Mar'] = '3,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Apr'] = '4,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'May'] = '5,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Jun'] = '6,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Jul'] = '7,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Aug'] = '8,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Sep'] = '9,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Oct'] = '10,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Nov'] = '11,' + datadf.Date.str[3:]
datadf.Date[datadf.Date.str[:3] == 'Dec'] = '12,' + datadf.Date.str[3:]

datadf.Date = pd.to_datetime(datadf.Date, format = '%m, %d, %Y')

TypeError: invalid type comparison

Write the dataframe to a csv

In [24]:
datadf.to_csv('earning_calls_data.csv', encoding  = 'utf-8')

In [38]:
html

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <title>Access to this page has been denied.</title>\n    <link href="https://fonts.googleapis.com/css?family=Open+Sans:300" rel="stylesheet">\n    <style>\n    html,body{\n      margin: 0;\n      padding: 0;\n      font-family: \'Open Sans\', sans-serif;\n      color: #000;\n    }\n\n    a{\n      color: #c5c5c5;\n      text-decoration: none;\n    }\n\n    .container{\n      align-items: center;\n      display: flex;\n      flex: 1;\n      justify-content: space-between;\n      flex-direction: column;\n      height: 100%;\n    }\n\n    .container > div {\n      width: 100%;\n      display: flex;\n      justify-content:center;\n    }\n\n    .container > div > div {\n      display: flex;\n      width: 80%;\n    }\n\n    .customer-logo-wrapper{\n      padding-top: 2rem;\n      flex-grow: 0;\n      background-color: #fff;\n      visibility: